# Example converting IDL code to Python using ChatGPT

_Author: Rebekah Esmaili, Science and Technology Corp (STC)_

This example contains functions used in a program searches a VIIRS DNB image find the closest point to a given latituide-longitude coordinate. We will explore two simpler functions, one that calculates the distance between two points (```realdistance```) and one that calculates the closest point when given an array of data (```areasearch```).

Below is an image of our example data. The two boxes are the Daynight Band (DNB) granule (in color) and the search point (in red). We want to find the closet pair of points.

![](test_data.png)

We will only need a few simple packages to run this example:

In [ ]:
import h5py
import numpy as np
import time

## Exercise 1: Calculate the Distance Between Points

We need to import two data files, which are from the [VIIRS Day/Night Band (DNB)](https://rammb.cira.colostate.edu/projects/alaska/blog/index.php/uncategorized/beginning-to-see-the-light-an-introduction-to-viirs-dnb-and-ncc/). The geolocation information is stored in the "GDNBO" file while the radiance is stored in the "SVBNB" file, so we need both to complete our exercises. The data in are [HDF5](https://www.hdfgroup.org/solutions/hdf5/) format.

In IDL, the code to import the data would look something like this:
```IDL
geofile = 'data/GDNBO_j01_d20230825_t0700543_e0702188_b29878_c20230825074232550000_oebc_ops.h5'
radfile = 'data/SVDNB_j01_d20230825_t0700543_e0702188_b29878_c20230825074447069000_oebc_ops.h5'

fid=h5f_open(geofile);
lonid=h5d_open(fid,'/All_Data/VIIRS-DNB-GEO_All/Longitude_TC');
lon=h5d_read(lonid) &  h5d_close,lonid
latid=h5d_open(fid,'/All_Data/VIIRS-DNB-GEO_All/Latitude_TC'); 
lat=h5d_read(latid) &  h5d_close,latid;
h5f_close,fid

fid=h5f_open(radfile);
radid=h5d_open(fid,'/All_Data/VIIRS-DNB-SDR_All/Radiance');
rad=h5d_read(radid) &  h5d_close,radid;
h5f_close,fid

```

Copy/paste the IDL code block above into [ChatGPT](https://chat.openai.com/) to import the sample data. We will use the two datasets it to test our functions.

**Solution**

The point that we'll search for is:

In [ ]:
targetlat = 53.0
targetlon = -78.0

As you can see, the dimensions are the same size in the test data. Now, lets use this data to calculate the distance between a two pairs of latitude-longitude coordinates. The function can handle both floats and arrays of floats.

In the code block below, rewrite the following IDL function using [ChatGPT](https://chat.openai.com/):

```IDL
FUNCTION realdistance,lat1,lon1,lat2,lon2
;return real earth distance in km
	PI=3.14159d
	temp=0.0d
	earth_radius=6378.0d ; //km
		;convert all to radians:
		tmplat1=double(lat1*PI/180.0);
		tmplon1=double(lon1*PI/180.0);
		tmplat2=double(lat2*PI/180.0);
		tmplon2=double(lon2*PI/180.0);

		temp=sin(tmplat1)*sin(tmplat2) + cos(tmplat1)*cos(tmplat2)*cos(tmplon2-tmplon1);
		temp=acos(temp);
		temp=temp*earth_radius;
return, temp;
END
```

***Solution:***


Now, let's test our function using our example data we imported earlier data (```lat1```, ```lon1```, ```lon1```, ```lon2```). 

Part of testing larger scripts is breaking them down into smaller bite sized peices and test/validate. Run the code block below to see if the function works correctly using the lat/lon arrays with three different values.

In [ ]:
distances = realdistance(lat_viirs[0,0], lon_viirs[0,0], targetlon, targetlat)

print("Coordinates:", lat_viirs[0,0], lon_viirs[0,0], targetlon, targetlat)
print("Distances:", distances, "km")

## Example 2: Get the radiance of pixels neighboring the closest point

Now that we have tested the distance function, let's convert an IDL function that uses ```realdistance```. The function below calculates the radiance at the point closest to the provided one. It's longer than the last one, so we'll break it down into part 1 and part 2.

```IDL
PRO areasearch, lat_viirs, lon_viirs, rad_viirs, targetlat, targetlon

    ; ============== Part 1 ==============

    ; to reduce looping, see if the granule even contains the point,
    ; if not, continue to next file
    ; Searching in quarter degree
    x=where(abs(lat_viirs-targetlat) LE 0.125)

    ; calculate the distance between the remaining points
    d=realdistance(lat_viirs(x),lon_viirs(x),targetlat,targetlon);

    ; check if any of the remaining points are within 1km of the 
    ; given point
    xind=where(d LE 1.0); 

    ; ============== Part 2 ==============

    ; The indices of all matches
    x1=x(xind);

    ; Find the max radiance value within the closest points
    mx=max(rad_viirs(x1),location); 
    ind= ARRAY_INDICES(rad_viirs, x1(location))

    I0=ind(0);
    J0=ind(1);
    rad0=mx;
    offset=1
    i1=I0-offset
    i2=I0+offset
    j1=j0-offset
    j2=j0+offset
    low=0	   
    high=768
    width=4064

	;make sure it is not on the edge:
	if ((i1 ge low) and (i1 lt width) and (i2 ge low) and (i2 lt width)$
             and (j1 ge low) and (j1 lt high) and (j2 ge low) and (j2 lt high)) then begin

        ;compute the total radiance within a rectangle around the brightest pixel
        radarea=total(rad(i1:i2,j1:j2))

        ;compute the min radiance within a rectangle around the brightest pixel
        minrad=Min(rad(i1:i2,j1:j2)) 
        
        ;compute the min radiance within a rectangle around the brightest pixel
        targetDist=realdistance(lat_viirs(I0,J0),lon_viirs(I0,J0),targetlat,targetlon);

		print,"Found."
		print, minrad, radarea, targetDist;

    endif
END


```

### Converting areasearch, Part 1

Copy/paste the code in Part 1 into ChatGPT. Enter the results below. I recommend not writing as a function just yet, we want to first test functionality.

**Solution**

The above code runs without any issues. Note that you'll have to remove conditionals/return statements.

### Benchmarking functions
The realdistance function would calculate the distances for all points in the lat_viirs and lon_viirs arrays, but by subsetting the arrays with x, the computation time is much faster. The time function in the time library to determine the start/end processing time in seconds. For example:

In [ ]:
start_time = time.time()
d = realdistance(lat_viirs, lon_viirs, targetlat, targetlon)
end_time = time.time()
print("Computing time:", end_time - start_time, "seconds")

start_time = time.time()
d = realdistance(lat_viirs[x], lon_viirs[x], targetlat, targetlon)
end_time = time.time()
print("Computing time:", end_time - start_time, "seconds")

It's generally a good idea to time your functions when converting from IDL to python. From above you can see the code that was subsetted was 3.5 times faster, primarilly due to fewer computations. While this speedup is small for one computation, if iterating over a large number of points (e.g., a global day), this can be significant.


### Converting areasearch, Part 2
Next, let's tackle part 2. This part finds the total and minimum radiance around the given point. Try to run the code as-is... does it work?

**Solution**

The code *techincally* works, but the distance is very far. So there was an index error.

The first error takes place in

```
# The indices of all matches
x1 = x[0][xind]

# Find the max radiance value within the closest points
mx = np.max(rad_viirs[x1])
location = np.argmax(rad_viirs[x1])
```

The above code fails at the ```mx = np.max(rad_viirs[x1])``` because x1 is a 1D index but rad_viirs is 2D.

### Commmon issue #1: ChatGPT incorrectly converts IDL expressions

Problem #1: ChatGPT incorrectly converted the IDL expression ```x1=x(xind);```. Here's a simple test. We want the index where the distance is the MIMIMUM value, right? We can check this. Let's print d[xind] and d[x1] and see if the distance is small. You'll see that xind correctly prints zeros (below zero km apart from the target point) whereas x1 is much further.

In [ ]:
print("distances (km) with xind :", d[xind])
print("distances (km) with x1:", d[x1])

In [ ]:
# Get the max value of the radiance in the search box:
mx = np.max(rad_viirs[x][xind])
location = np.argmax(rad_viirs[x][xind])

Update your code below with the fix:

**Solution**

There was no error, but there was also no output! The print statement is inside the if statement, so we need to see why it didn't trigger. Let's print the if statement:

In [ ]:
(low <= i1 < height), (low <= i2 < height), (low <= j1 < width), (low <= j2 < width)

We see the first condition failed, let's print the values:

In [ ]:
print(I0, J0)
print(i1, i2, j1, j2)

The points ```i1, i2, j1, j2``` are supposed to be the neighboring pixels next to the closet point.

From above, you'll see i1 is -1, which is not a permitted index in Python. The resulting indices are negative, and IDL has different index behavior than Python. "In IDL 8.0 however, a negative index is allowed, which will lead to the last element of the array being accessed, perhaps unintentionally, with no error thrown." In Python, an error is thrown. The if statement prevented this from happening, but if we removed it, there will be an error.

### Common Problem #2: ChatGPT incorrectly converted the index
ChatGPT incorrectly converted the index from unravel_indices. Similar to the problem #1, the returned indices are from the smaller, 1D array and not from the full-sized array.

* IDL's ```ARRAY_INDICES(Array, Index [, /DIMENSIONS] )``` function converts one-dimensional subscripts of an array into corresponding multi-dimensional subscripts.

* Python's ``np.unravel_index(indices, shape)`` function converts a flat index or array of flat indices into a tuple of coordinate arrays.

The fix is below, and it doesn't resemble the IDL code at all. We have to return to the purpose of the code: we want to find the index where the radiance is maximum within the closest points.

Below, we create a mask that has three conditions:
1. Determine index where radiance == maximum radiance
2. Determine index where latitude == latitude @ maximum radiance
3. Determine index where longitude == longitude @ maximum radiance

The resulting ```I0, J0``` are the corresponding indices in the original 2D array

In [ ]:
# Fix the neighboring points
mask = (rad_viirs == rad_viirs[x][xind][location]) & (lon_viirs == lon_viirs[x][xind][location]) & (lat_viirs == lat_viirs[x][xind][location])
I0, J0 = np.where(mask)[0].item(), np.where(mask)[1].item()

Let's check the output:

In [ ]:
i1=I0-offset
i2=I0+offset
j1=J0-offset
j2=J0+offset

print(i1, i2, j1, j2)

The rest of code works as expected. Let's put the pieces together:

**Solution**

Success! We calcualted the radiances at the closest point. The resulting VIIRS coordiantes (52.995228N, 77.991W) is within 1.09 km of the provided point.

## Best Practices

In conclusion, here are some recommendations and challenges illustrated by this example:

1. ChatGPT provides an excellent first guess of the code, but expect to refine the code.
2. Legacy code often requires some clean-up, build in time to remove usused code, unnecesary variables, etc.
3. Fast Python code avoids loops and uses optimized built-in functions, like those in NumPy.
4. Use test data to check if the output in Python matches in the output in IDL